In [6]:
from pathlib import Path
import pandas as pd
import altair as alt
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [7]:
Path.cwd()

PosixPath('/Users/ben/Developer/SpaceMissionDES/SpaceMissionDES/analysis')

#### Load the Data

In [8]:
data_dir = Path.cwd().parent.parent / "results" / "Parameter_Sweep" / "Case04-r05__Tx10-Tk02"

mc_results = pd.read_csv(data_dir / "mc.csv")
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,2,False,27.474903,2,12.500000,Tanker,Countdown
1,2,False,27.474903,3,22.883643,Tanker,Countdown
2,2,False,27.474903,4,27.474903,Tanker,RPOD
3,0,True,89.480727,2,25.500000,Tanker,Ascent
4,0,True,89.480727,3,41.318858,Tanker,Countdown


#### Summary Statistics

In [9]:
N = mc_results.replicant.max()
success_rate = sum(mc_results.groupby("replicant").first().outcome) / N
print(f"The success rate was: {success_rate} = {100*success_rate:.3f} %\n")

The success rate was: 0.8554759126521086 = 85.548 %



#### Mission Duration

In [10]:
df = (
    # mc_results[mc_results.outcome == True]
    mc_results
    .groupby("replicant", as_index=False)
    .first()
    .loc[:, ["replicant", "outcome", "duration"]]
)

df.loc[df.outcome == False, "duration"] = 1e6

alt.Chart(df).mark_bar().encode(
    # alt.X("duration:Q", bin=True),
    alt.X("duration:Q", bin=alt.Bin(extent=[50, 200], step=10)),
    y='count()',
).properties(
    width = 600
)

alt.Chart(...)

Compute the Emprical CDF

In [11]:
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

def plot_ECDF(df):
    sample = df.duration.values
    qe, pe = ecdf(sample)
    data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})
    fig = alt.Chart(data).mark_line().encode(
        x = alt.X("duration", scale=alt.Scale(domain=(0,200))),
        y = "prob"
    ).properties(width=500, height=250)
    return fig


In [12]:
fig = plot_ECDF(df)
fig.interactive()

alt.Chart(...)

In [13]:
fig = plot_ECDF(df)
fig.interactive()

alt.Chart(...)

In [14]:
df = (
    # mc_results[mc_results.outcome == True]
    mc_results
    .groupby("replicant", as_index=False)
    .first()
    .loc[:, ["replicant", "outcome", "duration"]]
)

df.loc[df.outcome == False, "duration"] = 1e6

df

,replicant,outcome,duration
0,0,True,89.480727
1,1,True,75.885828
2,2,False,1000000.000000
3,3,True,76.481949
4,4,False,1000000.000000
...,...,...,...
5995,5995,True,82.958913
5996,5996,True,82.958913
5997,5997,True,81.835911
5998,5998,True,87.348956


In [15]:
df

sample = df.duration.values
sample

# convert sample to a numpy array, if it isn't already
sample = np.atleast_1d(sample)

# find the unique values and their corresponding counts
quantiles, counts = np.unique(sample, return_counts=True)

# take the cumulative sum of the counts and divide by the sample size to
# get the cumulative probabilities between 0 and 1
cumprob = np.cumsum(counts).astype(np.double) / sample.size

#### Failure Diagnostics (Aggregate)

Which Vehicles Fail

In [16]:
# Relative failure frequency by vehicle
df = (
    mc_results
    .groupby("anomaly_vehicle")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_vehicle",
    x = "n"
)

alt.Chart(...)

Which Activities Fail?

In [17]:
# Relative failure frequency by activity
df = (
    mc_results
    .groupby("anomaly_activity")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_activity",
    x = "n"
)

alt.Chart(...)

#### Failure Diagnostics (per mission)

Avg ascent failures per mission

In [18]:
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,2,False,27.474903,2,12.500000,Tanker,Countdown
1,2,False,27.474903,3,22.883643,Tanker,Countdown
2,2,False,27.474903,4,27.474903,Tanker,RPOD
3,0,True,89.480727,2,25.500000,Tanker,Ascent
4,0,True,89.480727,3,41.318858,Tanker,Countdown


In [19]:
mc_results.query("anomaly_count > 1")
# df = mc_results.query("replicant == 3")
# df

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,2,False,27.474903,2,12.500000,Tanker,Countdown
1,2,False,27.474903,3,22.883643,Tanker,Countdown
2,2,False,27.474903,4,27.474903,Tanker,RPOD
3,0,True,89.480727,2,25.500000,Tanker,Ascent
4,0,True,89.480727,3,41.318858,Tanker,Countdown
...,...,...,...,...,...,...,...
23395,5996,True,82.958913,4,29.974903,Tanker,Countdown
23396,5996,True,82.958913,5,31.036659,Tanker,Countdown
23397,5996,True,82.958913,6,51.726783,Tanker,Countdown
23398,5998,True,87.348956,2,38.500000,Tanker,Countdown


In [20]:
# df.anomaly_activity.value_counts()

In [21]:
df = mc_results.groupby("replicant", as_index=False)["anomaly_activity"].value_counts()
df

,replicant,anomaly_activity,count
0,0,Countdown,2
1,0,Ascent,1
2,1,Countdown,1
3,2,Countdown,2
4,2,RPOD,1
...,...,...,...
7596,5997,Countdown,4
7597,5998,Ascent,1
7598,5998,Countdown,1
7599,5999,Countdown,6


In [22]:
# alt.Chart(df.unstack().Ascent.reset_index()).mark_boxplot().encode(x = "Ascent")

In [23]:
df = (
    mc_results
    # mc_results[mc_results.anomaly_activity != "Countdown"]
    .groupby("replicant", as_index=False)
    ["anomaly_activity"]
    .value_counts()
)

alt.Chart(df).mark_boxplot().encode(
    x = "count",
    y = "anomaly_activity"

)

alt.Chart(...)

In [24]:
df[df.anomaly_activity != "Countdown"]

,replicant,anomaly_activity,count
1,0,Ascent,1
4,2,RPOD,1
7,4,Return to Base,1
9,5,Ascent,1
15,10,Ascent,2
...,...,...,...
7582,5984,Ascent,1
7586,5988,Ascent,1
7593,5994,Return to Base,1
7597,5998,Ascent,1


In [25]:
mc_results

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,2,False,27.474903,2,12.500000,Tanker,Countdown
1,2,False,27.474903,3,22.883643,Tanker,Countdown
2,2,False,27.474903,4,27.474903,Tanker,RPOD
3,0,True,89.480727,2,25.500000,Tanker,Ascent
4,0,True,89.480727,3,41.318858,Tanker,Countdown
...,...,...,...,...,...,...,...
23395,5996,True,82.958913,4,29.974903,Tanker,Countdown
23396,5996,True,82.958913,5,31.036659,Tanker,Countdown
23397,5996,True,82.958913,6,51.726783,Tanker,Countdown
23398,5998,True,87.348956,2,38.500000,Tanker,Countdown


---
### Experiments

In [26]:
import pandas as pd
import altair as alt
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

sample = [1, 2, 2, 3, 3, 3, 4, 4, 5, 5,]
qe, pe = ecdf(sample)
data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})

alt.Chart(data).mark_line(interpolate='step-after', point=True, color="red").encode(
    x = alt.X("duration", axis=alt.Axis(values=[0, 1, 2, 3, 4, 5]), scale=alt.Scale(domain=(1,5))),
    y = "prob"
).properties(
    width=400, 
    height=400
)

alt.Chart(...)